<a href="https://colab.research.google.com/github/Investigator13th/CS246/blob/main/CS246_Colab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 4
## Collaborative Filtering

In [1]:
from IPython.display import Image
print("Colab 4 Mascot")
Image(url='https://media4.giphy.com/media/wtdVYmaRWJ1PyPcc8e/giphy.gif?cid=ecf05e478jdbzo92b89f2b5ud5184xda9cen1xce4bwmjeyu&ep=v1_stickers_search&rid=giphy.gif&ct=s',width=150)

Colab 4 Mascot


### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=ce1e171fdb0de1270bb30199600cbed813fb85b292bbe157886f88c5722966c3
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.6 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [7]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [8]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [9]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [10]:
''' 3 lines of code in total expected.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
print(f"num of training:{training.count()},  num of test:{test.count()}, num of movies:{items.count()}")

num of training:80000,  num of test:20000, num of movies:1682


Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

In [14]:
''' 5-6 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
# YOUR CODE HERE
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating",
          coldStartStrategy="drop", nonnegative=True)

model = als.fit(training)

Now compute the RMSE on the test dataset. **Please refer to Gradescope for the hyperparameters**.


In [13]:
''' 4-5 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
# YOUR CODE HERE
predictions = model.transform(test)

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error = {rmse}")

Root-mean-square error = 1.04595115949084


At this point, you can use the trained model to produce the top-K recommendations for each user.

In [16]:
''' 9-10 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
# YOUR CODE HERE
K = 10

userRecs = model.recommendForAllUsers(K)

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|      1|[{1643, 6.3455915...|
|      3|[{1512, 16.480536...|
|      5|[{787, 8.423346},...|
|      6|[{793, 5.632144},...|
|      9|[{1631, 7.165783}...|
+-------+--------------------+
only showing top 5 rows



Once you have working code for each cell above, **head over to Gradescope, carefully read the questions, and submit your solution for this Colab**! As a sanity check, the 4th most recommended movie under the setting of Q2.6 should be `Star Wars (1977)`.

In [23]:
userRecs.select('recommendations').take(1)

[Row(recommendations=[Row(item_id=1643, rating=6.3455915451049805), Row(item_id=1143, rating=5.947991847991943), Row(item_id=1639, rating=5.757406711578369), Row(item_id=114, rating=5.490695476531982), Row(item_id=1194, rating=5.477086067199707), Row(item_id=1642, rating=5.418614387512207), Row(item_id=936, rating=5.264247894287109), Row(item_id=1651, rating=5.246387481689453), Row(item_id=1650, rating=5.246387481689453), Row(item_id=1645, rating=5.246387481689453)])]

In [29]:
from pyspark.sql.types import ArrayType, IntegerType

def extract_item_ids(rows):
    return [row.item_id for row in rows]

extract_item_ids_udf = udf(extract_item_ids, ArrayType(IntegerType()))

movies_id_rec = userRecs.withColumn("recommendations", extract_item_ids_udf(col("recommendations"))).select("user_id", "recommendations")

In [30]:
movies_id_rec.take(5)

[Row(user_id=1, recommendations=[1643, 1143, 1639, 114, 1194, 1642, 936, 1651, 1650, 1645]),
 Row(user_id=3, recommendations=[1512, 960, 1021, 1093, 390, 536, 361, 1099, 695, 1529]),
 Row(user_id=5, recommendations=[787, 1631, 1425, 884, 853, 1394, 889, 753, 1005, 1368]),
 Row(user_id=6, recommendations=[793, 1193, 1286, 1512, 1558, 1643, 1203, 851, 1449, 1405]),
 Row(user_id=9, recommendations=[1631, 1400, 464, 1512, 1093, 695, 753, 733, 114, 1167])]

In [31]:
from pyspark.sql.functions import explode

# 展开recommendations列，使每一行对应一个item_id
exploded_movies_id_rec = movies_id_rec.select("user_id", explode("recommendations").alias("item_id"))

joined_df = exploded_movies_id_rec.join(items, on="item_id", how="inner")

selected_df = joined_df.select("user_id", "Movie")

# 使用group by和collect_list聚合函数重组结果
movies_name_rec = selected_df.groupBy("user_id").agg({"Movie": "collect_list"}).withColumnRenamed("collect_list(Movie)", "recommendations")

movies_name_rec.show()

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|      1|[Angel Baby (1995...|
|      2|[Angel Baby (1995...|
|      3|[World of Apu, Th...|
|      4|[Angel Baby (1995...|
|      5|[Roommates (1995)...|
|      6|[Crooklyn (1994),...|
|      7|[Mina Tannenbaum ...|
|      8|[Kicking and Scre...|
|      9|[Slingshot, The (...|
|     10|[8 Heads in a Duf...|
|     11|[8 Heads in a Duf...|
|     12|[Love! Valour! Co...|
|     13|[Mina Tannenbaum ...|
|     14|[Caught (1996), W...|
|     15|[8 Heads in a Duf...|
|     16|[8 Heads in a Duf...|
|     17|[Faust (1994), Go...|
|     18|[Angel Baby (1995...|
|     19|[8 Heads in a Duf...|
|     20|[Angel Baby (1995...|
+-------+--------------------+
only showing top 20 rows

